# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,grytviken,-54.2811,-36.5092,45.45,96,95,14.56,GS,1704579924
1,1,yazoo city,32.8551,-90.4056,41.92,73,100,9.01,US,1704579939
2,2,kiunga,-6.1219,141.2906,73.99,100,100,3.29,PG,1704579939
3,3,melfi,40.9965,15.6514,47.50,87,100,1.77,IT,1704579940
4,4,tiksi,71.6872,128.8694,-29.47,98,39,7.63,RU,1704579940


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 75) & (city_data_df["Max Temp"] < 85) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
narrow_city_df = narrow_city_df.dropna(how='any')
narrow_city_df.reset_index(inplace=True)
del narrow_city_df['index']

# Display sample data
narrow_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,49,viedma,-40.8135,-62.9967,77.81,28,0,10.13,AR,1704579953
1,111,lihue,21.9789,-159.3672,81.64,70,0,5.75,US,1704579972
2,172,wailua homesteads,22.0669,-159.3780,81.03,70,0,5.75,US,1704580000
3,213,kapa'a,22.0752,-159.3190,82.38,70,0,5.75,US,1704580018
4,316,limon,10.0000,-83.0333,82.29,74,0,8.05,CR,1704580053


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,49,viedma,-40.8135,-62.9967,77.81,28,0,10.13,AR,1704579953,
1,111,lihue,21.9789,-159.3672,81.64,70,0,5.75,US,1704579972,
2,172,wailua homesteads,22.0669,-159.3780,81.03,70,0,5.75,US,1704580000,
3,213,kapa'a,22.0752,-159.3190,82.38,70,0,5.75,US,1704580018,
4,316,limon,10.0000,-83.0333,82.29,74,0,8.05,CR,1704580053,
5,382,saint croix,17.7397,-64.7388,81.05,80,0,8.05,VI,1704580073,
6,393,brisas de zicatela,15.8369,-97.0419,84.72,58,0,6.91,MX,1704580075,
7,410,callao,-12.0667,-77.1500,75.49,69,0,18.41,PE,1704580080,
8,450,pisco,-13.7000,-76.2167,75.25,73,0,27.63,PE,1704580093,
9,507,pamandzi,-12.7967,45.2794,80.38,94,0,4.61,YT,1704580127,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude= row["Lat"]
    longitude= row["Lng"]
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    limit = "1"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()

    # Convert the API response to JSON format
    #commenting out so this is not shown (it's very long!)
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
viedma - nearest hotel: Níjar
lihue - nearest hotel: Kauai Palms
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
kapa'a - nearest hotel: Pono Kai Resort
limon - nearest hotel: Hotel NG
saint croix - nearest hotel: The Buccaneer
brisas de zicatela - nearest hotel: Casa de Olas
callao - nearest hotel: Casa Ronald
pisco - nearest hotel: La Portada
pamandzi - nearest hotel: Gite Le Madina


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,49,viedma,-40.8135,-62.9967,77.81,28,0,10.13,AR,1704579953,Níjar
1,111,lihue,21.9789,-159.3672,81.64,70,0,5.75,US,1704579972,Kauai Palms
2,172,wailua homesteads,22.0669,-159.3780,81.03,70,0,5.75,US,1704580000,Hilton Garden Inn Kauai Wailua Bay
3,213,kapa'a,22.0752,-159.3190,82.38,70,0,5.75,US,1704580018,Pono Kai Resort
4,316,limon,10.0000,-83.0333,82.29,74,0,8.05,CR,1704580053,Hotel NG
5,382,saint croix,17.7397,-64.7388,81.05,80,0,8.05,VI,1704580073,The Buccaneer
6,393,brisas de zicatela,15.8369,-97.0419,84.72,58,0,6.91,MX,1704580075,Casa de Olas
7,410,callao,-12.0667,-77.1500,75.49,69,0,18.41,PE,1704580080,Casa Ronald
8,450,pisco,-13.7000,-76.2167,75.25,73,0,27.63,PE,1704580093,La Portada
9,507,pamandzi,-12.7967,45.2794,80.38,94,0,4.61,YT,1704580127,Gite Le Madina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display
cityDataDF= hotel_df.loc[hotel_df["Hotel Name"] != "N/A"]

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    color = "City",
    tiles = "OSM",
    frame_width= 700,
    frame_height= 400,
    hover_cols= ["City", "Humidity","Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)